In [1]:
from bs4 import BeautifulSoup

from selenium import webdriver
import urllib.request

import time
import requests

In [36]:
url = 'https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno=20211125000655&docno=&viewerhost=&viewerport='

driver = webdriver.Chrome('C:/Users/JH/Desktop/chromedriver')
driver.get(url)

rink_box = []

rink = driver.find_element_by_id('docLocPath').get_attribute('value')
rink_box.append(rink)


'https://kind.krx.co.kr/external/2021/11/25/000655/20211125000924/91208.htm'

In [3]:
url = "https://kind.krx.co.kr/disclosure/details.do?method=searchDetailsMain#viewer"

driver = webdriver.Chrome('C:/Users/JH/Desktop/chromedriver')
driver.get(url)
driver.find_element_by_id('rKosdaq').click()
time.sleep(0.2)
driver.find_element_by_xpath('//*[@id="business"]/option[4]').click()
time.sleep(0.2)

driver.find_element_by_xpath('//*[@id="bfrDsclsType"]').click() # 공시유형 초기화 체크 해제
time.sleep(0.2)
driver.find_element_by_xpath('//*[@id="dsclsType01"]').click() # 수시공시 클릭
time.sleep(0.2)
driver.find_element_by_xpath('//*[@id="dsclsLayer01_67"]').click() # 횡령
time.sleep(0.2)
driver.find_element_by_xpath('//*[@id="currentPageSize"]/option[4]').click() # 100개씩 보기 선택
time.sleep(0.2)

elem1 = driver.find_element_by_name("fromDate")
elem2 = driver.find_element_by_name("toDate")
time.sleep(0.2)
elem1.clear()
time.sleep(0.2)
elem1.send_keys("2020-01-01")
time.sleep(0.2)
elem2.clear()
time.sleep(0.2)
elem2.send_keys("2020-12-31")
time.sleep(0.2)
driver.find_element_by_xpath('//*[@id="searchForm"]/section[1]/div/div[3]/a[1]').click() # 검색

open_code = []
time.sleep(1)
kk = int(driver.find_element_by_xpath('//*[@id="main-contents"]/section[2]/div[2]/em').text)
print(kk)
for i in range(kk):
    onclick_value = driver.find_element_by_xpath('//*[@id="main-contents"]/section[1]/table/tbody/tr[{0}]/td[4]/a'.format(i + 1)).get_attribute('onclick')
    onclick_value = onclick_value[18:32]
    print('tr 값 :', i + 1)
    time.sleep(0.3)
    open_code.append(onclick_value)

rink_box = []
for i in range(len(open_code)):
    url = 'https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno={0}&docno=&viewerhost=&viewerport='.format(int(open_code[i]))
    time.sleep(0.3)
    driver.get(url)
    time.sleep(0.3)
    rink = driver.find_element_by_id('docLocPath').get_attribute('value')
    rink_box.append(rink)


53
tr 값 : 1
tr 값 : 2
tr 값 : 3
tr 값 : 4
tr 값 : 5
tr 값 : 6
tr 값 : 7
tr 값 : 8
tr 값 : 9
tr 값 : 10
tr 값 : 11
tr 값 : 12
tr 값 : 13
tr 값 : 14
tr 값 : 15
tr 값 : 16
tr 값 : 17
tr 값 : 18
tr 값 : 19
tr 값 : 20
tr 값 : 21
tr 값 : 22
tr 값 : 23
tr 값 : 24
tr 값 : 25
tr 값 : 26
tr 값 : 27
tr 값 : 28
tr 값 : 29
tr 값 : 30
tr 값 : 31
tr 값 : 32
tr 값 : 33
tr 값 : 34
tr 값 : 35
tr 값 : 36
tr 값 : 37
tr 값 : 38
tr 값 : 39
tr 값 : 40
tr 값 : 41
tr 값 : 42
tr 값 : 43
tr 값 : 44
tr 값 : 45
tr 값 : 46
tr 값 : 47
tr 값 : 48
tr 값 : 49
tr 값 : 50
tr 값 : 51
tr 값 : 52
tr 값 : 53


In [4]:
rink_box

['https://kind.krx.co.kr/external/2020/11/19/000166/20201119000357/70057.htm',
 'https://kind.krx.co.kr/external/2020/11/13/001299/20201112001383/70008.htm',
 'https://kind.krx.co.kr/external/2020/11/03/000224/20201030000401/70057.htm',
 'https://kind.krx.co.kr/external/2020/11/02/000574/20201102001077/70057.htm',
 'https://kind.krx.co.kr/external/2020/10/22/000320/20201022000518/70008.htm',
 'https://kind.krx.co.kr/external/2020/10/20/000509/20201020001277/70057.htm',
 'https://kind.krx.co.kr/external/2020/10/13/000487/20201013001080/70008.htm',
 'https://kind.krx.co.kr/external/2020/10/07/000511/20201006000755/70008.htm',
 'https://kind.krx.co.kr/external/2020/09/22/000433/20200921000541/70020.htm',
 'https://kind.krx.co.kr/external/2020/09/16/000244/20200831000599/70057.htm',
 'https://kind.krx.co.kr/external/2020/09/01/000471/20200831001339/70008.htm',
 'https://kind.krx.co.kr/external/2020/08/20/000489/20200820000546/70057.htm',
 'https://kind.krx.co.kr/external/2020/08/04/000435/

In [6]:
from bs4 import BeautifulSoup
import urllib.request

In [7]:
html = urllib.request.urlopen('https://kind.krx.co.kr/external/2020/11/19/000166/20201119000357/70057.htm')

soup = BeautifulSoup(html, 'html.parser').text

In [11]:
new_box = []

new_box.append(soup)
new_box

['\n\n\n\n\n\n:: 70057_횡령ㆍ배임 혐의 진행사항\n\n\n\n\n\n\n\n\n횡령ㆍ배임 혐의 진행사항\n\n\n\n\n\n\n\n\n1. 사고발생내용\n\n\n1. 당사의 前대표이사 류○○에 대한 서울중앙지방검찰청에서 2018년 9월 12일 공소 제기한 사건으로1심 판결에 대한 3심(상고심) 판결 내용을 공시 규정에 따라 공시합니다.\n         \n가. 대상자\n         \xa0- 前대표이사 류○○\n         \n나. 특정경제범죄가중처벌등에 관한 법률위반(횡령)\n         \n다. 자본시장과금융투자업에 관한 법률위반\n\n\n\n\n2. 횡령 등 관련사항\n\n\n발생금액(원)\n\n\n6,000,000,000\n\n\n\n\n자기자본(원)\n\n\n52,296,857,633\n\n\n\n\n자기자본대비(%)\n\n\n11.47\n\n\n\n\n대기업해당여부\n\n\n미해당\n\n\n\n\n최초 공시일자\n\n\n2018-09-18\n\n\n\n\n3. 진행사항\n\n\n1. 대법원 3심(상고심) 판결\n         \n\xa0\xa0가. 판결선고 : 2020.10.23.\n         \xa0\xa0나. 판결 내용\n         \xa0\xa0 \xa0 \xa0\n         \xa0\xa0 \xa0 \xa0- 상고를 기각한다.\n         \n\n2. 서울고등법원 2심(항소심) 판결\n         \n\xa0\xa0가. 판결선고: 2020.08.13\n         \xa0\xa0나. 판결 내용\n         \xa0\xa0 \xa0 \xa0- 대상자 : 前대표이사 류○○\n         \xa0\xa0 \xa0\n         \xa0\xa0 \xa0 \xa0- 특정경제범죄가중처벌 등에 관한 법률\n         \xa0\xa0 \xa0 \xa0 \xa0위반(횡령)\n         \n\xa0\xa0 \xa0 \xa0- 피고인 류○○의 항소와 검사의 항소를\n       

In [9]:
from nltk.tokenize import word_tokenize

word_tokenize(soup)

[':',
 ':',
 '70057_횡령ㆍ배임',
 '혐의',
 '진행사항',
 '횡령ㆍ배임',
 '혐의',
 '진행사항',
 '1',
 '.',
 '사고발생내용',
 '1',
 '.',
 '당사의',
 '前대표이사',
 '류○○에',
 '대한',
 '서울중앙지방검찰청에서',
 '2018년',
 '9월',
 '12일',
 '공소',
 '제기한',
 '사건으로1심',
 '판결에',
 '대한',
 '3심',
 '(',
 '상고심',
 ')',
 '판결',
 '내용을',
 '공시',
 '규정에',
 '따라',
 '공시합니다',
 '.',
 '가',
 '.',
 '대상자',
 '-',
 '前대표이사',
 '류○○',
 '나',
 '.',
 '특정경제범죄가중처벌등에',
 '관한',
 '법률위반',
 '(',
 '횡령',
 ')',
 '다',
 '.',
 '자본시장과금융투자업에',
 '관한',
 '법률위반',
 '2',
 '.',
 '횡령',
 '등',
 '관련사항',
 '발생금액',
 '(',
 '원',
 ')',
 '6,000,000,000',
 '자기자본',
 '(',
 '원',
 ')',
 '52,296,857,633',
 '자기자본대비',
 '(',
 '%',
 ')',
 '11.47',
 '대기업해당여부',
 '미해당',
 '최초',
 '공시일자',
 '2018-09-18',
 '3',
 '.',
 '진행사항',
 '1',
 '.',
 '대법원',
 '3심',
 '(',
 '상고심',
 ')',
 '판결',
 '가',
 '.',
 '판결선고',
 ':',
 '2020.10.23',
 '.',
 '나',
 '.',
 '판결',
 '내용',
 '-',
 '상고를',
 '기각한다',
 '.',
 '2',
 '.',
 '서울고등법원',
 '2심',
 '(',
 '항소심',
 ')',
 '판결',
 '가',
 '.',
 '판결선고',
 ':',
 '2020.08.13',
 '나',
 '.',
 '판결',
 '내용',
 '-',
 '대상자',
 ':',
 '前대표이사',